<a href="https://colab.research.google.com/github/shreya-1x1/Iris-classification-/blob/main/07_advanced_rag_with_llama_3_in_langchain_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import textwrap
from pathlib import Path
from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-api-core 2.16.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-aiplatform 1.59.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but y

In [3]:
!mkdir data
!gdown 1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9 -O "data/meta-earnings.pdf"

Downloading...
From: https://drive.google.com/uc?id=1ee-BhQiH-S9a2IkHiFbJz9eX_SfcZ5m9
To: /content/data/meta-earnings.pdf
100% 160k/160k [00:00<00:00, 87.9MB/s]


## Document Parsing

In [4]:
instruction = """The provided document is Meta First Quarter 2024 Results.
This form provides detailed financial information about the company's performance for a specific quarter.
It includes unaudited financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""

parser = LlamaParse(
    api_key=userdata.get("LLAMA_PARSE"),
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("./data/meta-earnings.pdf")

Started parsing the file under job_id cac11eca-8e8c-4aa1-9c25-53294497b32b


In [5]:
parsed_doc = llama_parse_documents[0]

In [6]:
Markdown(parsed_doc.text[:4096])

# Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

# First Quarter 2024 Financial Highlights

|In millions, except percentages and per share amounts|Three Months Ended March 31,| |2024| |2023|% Change|
|---|---|---|---|---|---|---|
|Revenue|$36,455|$28,645|27%| | | |
|Costs and expenses|$22,637|$21,418|6%| | | |
|Income from operations|$13,818|$7,227|91%| | | |
|Operating margin|38%|25%| | | | |
|Provision for income taxes|$1,814|$1,598|14%| | | |
|Effective tax rate|13%|22%| | | | |
|Net income|$12,369|$5,709|117%| | | |
|Diluted earnings per share (EPS)|$4.71|$2.20|114%| | | |

# First Quarter 2024 Operational and Other Financial Highlights

- Family daily active people (DAP) – DAP was 3.24 billion on average for March 2024, an increase of 7% year-over-year.
- Ad impressions – Ad impressions delivered across our Family of Apps increased by 20% year-over-year.
- Average price per ad – Average price per ad increased by 6% year-over-year.
- Revenue – Total revenue and revenue on a constant currency basis were $36.46 billion and $36.35 billion, respectively, both of which increased by 27% year-over-year.
- Costs and expenses – Total costs and expenses were $22.64 billion, an increase of 6% year-over-year.
- Capital expenditures – Capital expenditures, including principal payments on finance leases, were $6.72 billion.
- Capital return program – Share repurchases were $14.64 billion of our Class A common stock and dividends payments were $1.27 billion.
- Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were $58.12 billion as of March 31, 2024. Free cash flow was $12.53 billion.
- Headcount – Headcount was 69,329 as of March 31, 2024, a decrease of 10% year-over-year.
---
CFO Outlook Commentary

We expect second quarter 2024 total revenue to be in pe range of $36.5-39 billion. Our guidance assumes foreign currency is a 1% headwind to year-over-year total revenue growp, based on current exchange rates.
We expect full-year 2024 total expenses to be in pe range of $96-99 billion, updated from our prior outlook of $94-99 billion due to higher infrastructure and legal costs. For Reality Labs, we continue to expect operating losses to increase meaningfully year-over-year due to our ongoing product development efforts and our investments to furper scale our ecosystem.
We anticipate our full-year 2024 capital expenditures will be in pe range of $35-40 billion, increased from our prior range of $30-37 billion as we continue to accelerate our infrastructure investments to support our artificial intelligence (AI) roadmap. While we are not providing guidance for years beyond 2024, we expect capital expenditures will continue to increase next year as we invest aggressively to support our ambitious AI research and product development efforts.
Absent any changes to our tax landscape, we expect our full-year 2024 tax rate to be in pe mid-teens.
In addition, we continue to monitor an active regulatory landscape, including pe increasing legal and regulatory headwinds in pe EU and pe U.S. pat could significantly impact our business and our financial results.
Q1 was a good start to pe year. We're seeing strong momentum wipin our Family of Apps and are making important progress on our longer-term AI and Reality Labs initiatives pat have pe potential to transform pe way people interact wip our services over pe coming years.
---
Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnin

In [7]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

## Vector Embeddings

In [8]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

11

In [10]:
print(docs[0].page_content)

Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

First Quarter 2024 Financial Highlights

In millions, except percentages and per share amounts Three Months Ended March 31, 2024 2023 % Change Revenue $36,455 $28,645 27% Costs and expenses $22,637 $21,418 6% Income from operations $13,818 $7,227 91% Operating margin 38% 25% Provision for income taxes $1,814 $1,598 14% Effective tax rate 13% 22% Net income $12,369 $5,709 117% Diluted earnings per share (EPS) $4.71 $2.20 114%

First Quarter 2024 Operational and Other Financial Highlights

Family daily active peo

In [11]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [12]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [13]:
%%time
query = "What is the most important innovation from Meta?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 472 ms, sys: 2.6 ms, total: 475 ms
Wall time: 514 ms


In [14]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

score: 0.6154119568600498
--------------------------------------------------------------------------------

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

score: 0.5717282420622319
--------------------------------------------------------------------------------

text: Reconciliation of cash, cash equivalents, and restricted cash to the condensed consolidated balance sheets

Cash and cash equivalents $32,307 $11,551 Restricted cash, included in prepaid expenses and other current assets 84 224 Restricted cash, inclu

In [15]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 314 ms, sys: 54.1 ms, total: 368 ms
Wall time: 371 ms


In [16]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

id: 9f4d9ea3eba14dbe997108bae8126d3a

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

--------------------------------------------------------------------------------

id: 076b0537c1cc4703a856fffa97e79fe3

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

--------------------------------------------------------------------------------

id: 1cd029ba0af445149765451e27db410a

text: Reconciliation of cash, cash equivalents, and restricted cash to the condensed consolidated balance sheets

Cash and cash equivalents $32,307 $11,551 Restricted cash, included in prepaid e

## Reranking

In [17]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 78.2MiB/s]


In [18]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 2.84 s, sys: 201 ms, total: 3.04 s
Wall time: 3.04 s


3

In [19]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: 9f4d9ea3eba14dbe997108bae8126d3a

text: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta foun

score: 0.151275634765625
--------------------------------------------------------------------------------

id: 076b0537c1cc4703a856fffa97e79fe3

text: Webcast and Conference Call Information

Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of Meta's earnings conference call can be accessed at investor.fb.com, along with the earnings press rel

score: 0.011639031581580639
--------------------------------------------------------------------------------

id: 40844b37f51a42b587aa64675c3c6ecf

text: META PLATFORMS, INC.

CONDENSED CONSOLIDATED BALANCE SHEETS

March 31, 2024 December 31, 2023 Assets Current assets: Cash and cash equi

## Q&A Over Document

In [20]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [21]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [23]:
%%time
response = qa.invoke("What is the most significant innovation from Meta?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Meta Reports First Quarter 2024 Results

MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024.

"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."

First Quarter 2024 Financial Highlights

In millions, except percentages and per share amounts Three Months Ended March 31, 2024 2023 % Change Revenue $36,455 $28,645 27% Costs and expenses $22,637 $21,418 6% 

In [24]:
print_response(response)

Based on the provided information, the most significant innovation from Meta is the new version of
Meta AI with Llama 3, which is mentioned in the quote from Mark Zuckerberg, Meta founder and CEO.
This suggests that Meta is making progress in building the world's leading AI.

Additionally, the information highlights Meta's focus on building the metaverse, which is mentioned
as one of the areas where the company is making steady progress. However, it does not provide
further details on what specific innovations or developments are being made in this area.


In [25]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)

In [26]:
%%time
response = qa.invoke("What is the revenue for 2024 and % change?")

Running pairwise ranking..
CPU times: user 3.87 s, sys: 7.85 ms, total: 3.88 s
Wall time: 4.63 s


In [28]:
Markdown(response["result"])

The revenue for 2023 is not explicitly stated in the provided information. However, we can infer it from the "First Quarter 2024 Financial Highlights" section, which mentions that the revenue for Q1 2024 is $36,455 million, with a 27% year-over-year increase. This implies that the revenue for Q1 2023 was $28,645 million.

Here's the relevant information:

* Revenue (Q1 2024): $36,455 million
* Year-over-year increase: 27%
* Revenue (Q1 2023): $28,645 million

In [29]:
%%time
response = qa.invoke("What is the revenue for 2023?")

Running pairwise ranking..
CPU times: user 2.96 s, sys: 6.65 ms, total: 2.96 s
Wall time: 4.22 s


In [30]:
print_response(response)

The revenue for 2023 is not explicitly stated in the provided information. However, we can infer it
from the "First Quarter 2024 Financial Highlights" section, which mentions that the revenue for Q1
2024 is $36,455 million, with a 27% year-over-year increase. This implies that the revenue for Q1
2023 was $28,645 million.

Here's the relevant information:

* Revenue (Q1 2024): $36,455 million
* Year-over-year increase: 27%
* Revenue (Q1 2023): $28,645 million


In [31]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2024? Calculate the answer"
)

Running pairwise ranking..
CPU times: user 2.63 s, sys: 22 ms, total: 2.66 s
Wall time: 4.64 s


In [32]:
print_response(response)

Based on the provided information, we can calculate the revenue minus costs and expenses for 2024 as
follows:

**Revenue:**
The CFO Outlook Commentary mentions that the company expects second-quarter 2024 total revenue to be
in the range of $36.5-39 billion. However, this is only for the second quarter, and we need the
full-year revenue. Unfortunately, the full-year revenue guidance is not provided. Therefore, we
cannot calculate the exact revenue minus costs and expenses for 2024.

**Costs and Expenses:**
The CFO Outlook Commentary mentions that the company expects full-year 2024 total expenses to be in
the range of $96-99 billion.

Since we don't have the full-year revenue guidance, we cannot calculate the exact revenue minus
costs and expenses for 2024. However, we can provide some insights:

* If we assume the midpoint of the revenue range for the second quarter ($37.75 billion) and
extrapolate it to the full year (assuming the same growth rate), we can estimate the full-year
reven

In [33]:
%%time
response = qa.invoke(
    "How much is the revenue minus the costs and expenses for 2023? Calculate the answer"
)

Running pairwise ranking..
CPU times: user 3.62 s, sys: 14.4 ms, total: 3.63 s
Wall time: 5.54 s


In [34]:
print_response(response)

To calculate the revenue minus the costs and expenses for 2023, we need to find the total revenue
and total income from operations for 2023.

From the Segment Information section, we can see that the total revenue for 2023 is $28,645.

From the same section, we can see that the total income from operations for 2023 is $7,227.

However, the income from operations is not the same as revenue minus costs and expenses. To find the
costs and expenses, we need to look at the Condensed Consolidated Statements of Cash Flows.

From the Condensed Consolidated Statements of Cash Flows, we can see that the net income for 2023 is
$5,709. To find the costs and expenses, we can subtract the net income from the total revenue:

Costs and expenses = Total revenue - Net income
= $28,645 - $5,709
= $22,936

So, the revenue minus the costs and expenses for 2023 is $7,227 (income from operations) or $5,709
(net income), depending on how you define "costs and expenses".

Additional helpful information:

* The

In [35]:
%%time
response = qa.invoke("What is the expected revenue for the second quarter of 2024?")

Running pairwise ranking..
CPU times: user 3.6 s, sys: 13.9 ms, total: 3.62 s
Wall time: 4.51 s


In [36]:
Markdown(response["result"])

**Answer:** The expected revenue for the second quarter of 2024 is in the range of $36.5-39 billion.

**Additional helpful information:**

* The company's guidance assumes a 1% headwind to year-over-year total revenue growth due to foreign currency exchange rates.
* The company has reported a strong start to the year, with revenue increasing by 27% year-over-year in the first quarter of 2024.

In [37]:
%%time
response = qa.invoke("What is the overall outlook of Q1 2024?")

Running pairwise ranking..
CPU times: user 3.55 s, sys: 4.86 ms, total: 3.56 s
Wall time: 4.93 s


In [38]:
print_response(response)

**Overall Outlook of Q1 2024:**

The overall outlook of Q1 2024 is positive. According to Mark Zuckerberg, Meta's founder and CEO,
"It's been a good start to the year." The company reported strong financial results, with revenue
increasing by 27% year-over-year to $36.46 billion, and net income increasing by 117% year-over-year
to $12.37 billion.

**Additional Helpful Information:**

* Family daily active people (DAP) increased by 7% year-over-year to 3.24 billion on average for
March 2024.
* Ad impressions delivered across the Family of Apps increased by 20% year-over-year.
* Average price per ad increased by 6% year-over-year.
* The company saw strong momentum in its Family of Apps and made important progress on its longer-
term AI and Reality Labs initiatives.


## References

- [Meta Reports First Quarter 2024 Results](https://s21.q4cdn.com/399680738/files/doc_financials/2024/q1/Meta-03-31-2024-Exhibit-99-1_FINAL.pdf)